In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
import sys
sys.path.append('../..')

from PIL import Image
from segmentation_tools.visualisation import draw_sem_seg, VisImage
from segmentation_tools.label_data import get_ade150, get_nyu40, get_scannet_all, get_wordnet, get_replica
from segmentation_tools.label_mappings import set_ids_according_to_names, \
        match_scannet_ade150, \
        match_scannet_nyu40, \
        match_ade150_nyu40, \
        match_scannet_wordnet199, \
        LabelMatcher, \
        set_colors
from pathlib import Path
import skimage

# scene 42445991
# frames:
# sdfstudio_path: pred_sdfstudio_2023-08-02_230529

# scene 42897688
# frames: 0, 74
# sdfstudio_path: pred_sdfstudio_2023-08-02_230607

scene = 42897688
key = 50
sdfstudio_path = 'pred_sdfstudio_2023-08-02_230607'

scene_dir = Path(f'/home/weders/scratch/scratch/scannetter/arkit/raw/Validation/{scene}')
img = cv2.imread(f'{scene_dir}/color/{key}.jpg')[..., ::-1]

# label = cv2.imread(f'{scene_dir}/label-filt/{key}.png',
#                    cv2.IMREAD_UNCHANGED)
# our_label = cv2.imread(f'{scene_dir}/label_agile3d/{key}.png', cv2.IMREAD_UNCHANGED)

consensus = cv2.imread(f'{scene_dir}/pred_consensus_noscannet_new/{key}.png',
                   cv2.IMREAD_UNCHANGED)
# sdfstudio = cv2.imread(f'{scene_dir}/{sdfstudio_path}/{(key):05d}.png',
#                    cv2.IMREAD_UNCHANGED)



classid2wn = {x['id']: x['name'] for x in get_wordnet(label_key='wn199-merged-v2')}
scannet_id_to_name = {x['id'] : x['name'] for x in get_scannet_all()}
scannet_id_to_color = {x['id'] : x['color'] for x in get_scannet_all()}


keys = sorted(
            int(x.name.split('.')[0])
            for x in (scene_dir / 'color_old').iterdir())
label_template = 'label-filt/{k}.png'
label_space = 'id'


vis_labelmaker = VisImage(img)
draw_sem_seg(sdfstudio, vis_labelmaker, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])

vis_consensus = VisImage(img)
draw_sem_seg(consensus, vis_consensus, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])


_, plots = plt.subplots(1, 2, figsize=(20, 10))

plots[0].imshow(img)
plots[0].axis('off')
plots[0].set_title('LabelMaker3D')


# plots[1].imshow(vis_labelmaker.get_image())
# plots[1].axis('off')
# plots[1].set_title('LabelMaker3D')

plots[1].imshow(vis_consensus.get_image())
plots[1].axis('off')
plots[1].set_title('Consensus')
plt.show()


plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.gca().axis('off')
plt.show()

# plt.figure(figsize=(20, 10))
# plt.imshow(vis_labelmaker.get_image())
# plt.gca().axis('off')
# plt.show()

plt.figure(figsize=(20, 10))
plt.imshow(vis_consensus.get_image())
plt.gca().axis('off')
plt.show()



In [ ]:
# iterate over all frames for scene

scene = '42897688'
scene_dir = Path(
    f'/home/weders/scratch/scratch/scannetter/arkit/raw/Validation/{scene}')
# sdfstudio_pred = 'pred_sdfstudio_2023-08-02_230529'
# sdfstudio_pred = 'pred_sdfstudio_2023-08-02_230607'
keys = sorted([
    int(k.split('/')[-1].replace('.jpg', ''))
    for k in os.listdir((scene_dir / 'color'))
])
# keys_rendering = sorted([int(k.split('/')[-1].replace('.png', '')) for k in os.listdir((scene_dir / sdfstudio_pred))])

rgb_keys = sorted([
    round(float(k.split('_')[-1].replace('.png', '')), 3)
    for k in os.listdir((scene_dir / 'vga_wide'))
])
depth_keys = sorted([
    round(float(k.split('_')[-1].replace('.png', '')), 3)
    for k in os.listdir((scene_dir / 'highres_depth'))
])
print(depth_keys)


def get_rgb_key(depth_key):
  for cj, k in enumerate(rgb_keys):
    if k >= depth_key:
      key_before = rgb_keys[cj - 1]
      key_after = k
      break

  delta_before = abs(key_before - depth_key)
  delta_after = abs(key_after - depth_key)

  if delta_before >= delta_after:
    return key_after
  else:
    return key_before


for idx, key in enumerate(keys):
  print(idx, key)

  # rgb_key = get_rgb_key(depth_keys[idx])
  # print(round(rgb_key, 3))
  if idx % 2 != 0:
    continue

#     # print(f'{scene}_{rgb_key:.3f}.jpg')
#     key_rendering = keys_rendering[idx // 2]

  img = cv2.imread(f'{scene_dir}/color/{key}.jpg')[..., ::-1]
  # img = cv2.imread(f'{scene_dir}/vga_wide/{scene}_{rgb_key:.3f}.png')[..., ::-1]

  #     label = cv2.imread(f'{scene_dir}/label-filt/{key}.png',
  #                        cv2.IMREAD_UNCHANGED)
  #     our_label = cv2.imread(f'{scene_dir}/label_agile3d/{key}.png', cv2.IMREAD_UNCHANGED)

  consensus = cv2.imread(f'{scene_dir}/pred_consensus_noscannet_new/{key}.png',
                         cv2.IMREAD_UNCHANGED)
  #     sdfstudio = cv2.imread(f'{scene_dir}/{sdfstudio_pred}/{(key):05d}.png',
  #                        cv2.IMREAD_UNCHANGED)

  classid2wn = {x['id']: x['name'] for x in get_wordnet()}
  scannet_id_to_name = {x['id']: x['name'] for x in get_scannet_all()}
  scannet_id_to_color = {x['id']: x['color'] for x in get_scannet_all()}

  classes = [
      x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'),
                                key=lambda x: x['id'])
  ]

  keys = sorted(
      int(x.name.split('.')[0]) for x in (scene_dir / 'color').iterdir())
  label_template = 'label-filt/{k}.png'
  label_space = 'id'
  #     plt.figure(figsize=(20, 10))
  #     plt.imshow(img)
  #     plt.gca().axis('off')
  #     plt.show()
  _, plots = plt.subplots(1, 3, figsize=(40, 10))

  plots[0].imshow(img)
  plots[0].axis('off')
  plots[0].set_title('Image')

  #     vis = VisImage(img)
  #     draw_sem_seg(our_label, vis,
  #                  classes={x['id']: x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])},
  #                  colors=[x['color'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])])
  #     plots[1].imshow(vis.get_image())
  #     plots[1].axis('off')
  #     plots[1].set_title('Ground Truth')
  #     vis = VisImage(img)
  #     draw_sem_seg(label, vis,
  #                  classes=[scannet_id_to_name[i] if i in scannet_id_to_name else 'unknown' for i in range(2000)],
  #                  colors=[scannet_id_to_color[i] if i in scannet_id_to_name else [0, 0, 0] for i in range(2000)],)
  #     plots[2].imshow(vis.get_image())
  #     plots[2].axis('off')
  #     plots[2].set_title('ScanNet')
  vis = VisImage(img)
  draw_sem_seg(consensus,
               vis,
               classes=[
                   x['name']
                   for x in sorted(get_wordnet(label_key='wn199-merged-v2'),
                                   key=lambda x: x['id'])
               ],
               colors=[
                   x['color']
                   for x in sorted(get_wordnet(label_key='wn199-merged-v2'),
                                   key=lambda x: x['id'])
               ])
  plots[1].imshow(vis.get_image())
  plots[1].axis('off')
  plots[1].set_title('LabelMaker3D')
  vis = VisImage(img)
  # draw_sem_seg(consensus, vis,
  #             classes=[x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])],
  #             colors=[x['color'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])])
  #plots[2].imshow(vis.get_image())
  #plots[2].axis('off')
  #plots[2].set_title('Consensus')
  #plt.tight_layout()
  plt.show()


In [ ]:
# colorize labelmaker arkit pointcloud
# iterate over all frames for scene

import open3d as o3d

scenes = ['42445991', '42446527', '42897688']
os.makedirs('colored_meshes', exist_ok=True)
for sc in scenes:
  scene_dir = Path(
      f'/home/weders/scratch/scratch/scannetter/arkit/raw/Validation/{sc}')

  label_file = next(
      iter([f for f in scene_dir.iterdir() if 'labels_3d' in str(f)]))
  mesh_file = next(iter([f for f in scene_dir.iterdir() if '.ply' in str(f)]))
  mesh = o3d.io.read_triangle_mesh(str(mesh_file))

  labels_3d = np.loadtxt(label_file)
  mesh_colors = np.asarray(mesh.vertex_colors)
  colors = np.zeros_like(mesh_colors)

  id_to_color = {
      x['id']: x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])
  }

  for l in np.unique(labels_3d):
    colors[labels_3d == l] = id_to_color[int(l)]

  colors = colors / 255.
  mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
  o3d.io.write_triangle_mesh(f'colored_meshes/{sc}_label_color.ply', mesh)


In [ ]:
# colorize mask3d meshes
scenes = ['42445991', '42446527', '42897688']

for sc in scenes:
  scene_dir = Path(
      f'/home/weders/scratch/scratch/scannetter/arkit/raw/Validation/{sc}')
  mask3d_dir = scene_dir / 'pred_mask3d'

  pred_file = next(
      iter([f for f in mask3d_dir.iterdir() if '3dod_mesh.txt' in str(f)]))

  mask_paths = []
  confidences = []
  classes = []

  with open(pred_file, 'r') as file:
    for line in file:
      m_file, l, c = line.rstrip().split(' ')
      mask_paths.append(m_file)
      classes.append(int(l))
      confidences.append(float(c))

  sorting_indices = np.argsort(np.asarray(confidences))[::-1]

  mesh_file = next(iter([f for f in scene_dir.iterdir() if '.ply' in str(f)]))
  mesh = o3d.io.read_triangle_mesh(str(mesh_file))
  mesh_colors = np.asarray(mesh.vertex_colors)
  colors = np.zeros_like(mesh_colors)
  colored_mask = np.zeros_like(colors[:, 0])
  id_to_color = {
      x['id']: x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])
  }

  for idx in sorting_indices:
    m = np.loadtxt(mask3d_dir / mask_paths[idx])
    l = classes[idx]

    m = (m == 1) & (colored_mask == 0)
    colored_mask[m] = 1

    colors[m] = scannet_id_to_color[l]

  colors = colors / 255.
  mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
  o3d.io.write_triangle_mesh(f'colored_meshes/{sc}_mask3d_color.ply', mesh)
